<a href="https://colab.research.google.com/github/rosafilgueira/PyCodeSearch/blob/main/Registry_search.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Deep Learning Transformer models for code


## UnixCoder RepoSIM

It uses Lazyhope/unixcoder-nine-advtest - which has been trained following: https://github.com/microsoft/CodeBERT/blob/master/UniXcoder/downstream-tasks/code-search/README.md

This model (which uses a bi-enconder approach) works very well for both - code similarity and text similarity.

In [1]:
!pip install tensorflow
!pip install -U accelerate
!pip install docarray
!pip install pandas
!pip install torch
!pip install transformers
!pip install tqdm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 227.6/227.6 kB 5.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 233.2/233.2 kB 9.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 138.7/138.7 kB 12.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 65.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 29.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 84.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 65.8 MB/s eta 0:00:00


In [6]:
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

## Add database example

In [2]:
import pandas as pd

# Define the codes and their corresponding docstrings
codes = [
    "def add(a, b):\n    return a + b",
    "def subtract(a, b):\n    return a - b",
    "def multiply(a, b):\n    return a * b",
    "def divide(a, b):\n    return a / b",
    "def power(a, b):\n    return a ** b",
    "def modulus(a, b):\n    return a % b"
]

docs = [
    "This function adds two numbers.",
    "This function subtracts the second number from the first.",
    "This function multiplies two numbers.",
    "This function divides the first number by the second.",
    "This function raises the first number to the power of the second.",
    "This function returns the remainder when the first number is divided by the second."
]

# Create the dataframe
registry = pd.DataFrame({
    'code': codes,
    'doc': docs
})

### dataframe
registry

,code,doc
0,"def add(a, b):\n return a + b",This function adds two numbers.
1,"def subtract(a, b):\n return a - b",This function subtracts the second number from...
2,"def multiply(a, b):\n return a * b",This function multiplies two numbers.
3,"def divide(a, b):\n return a / b",This function divides the first number by the ...
4,"def power(a, b):\n return a ** b",This function raises the first number to the p...
5,"def modulus(a, b):\n return a % b",This function returns the remainder when the f...


## Load the model


In [3]:
from transformers import pipeline

model = pipeline(
    model="Lazyhope/RepoSim",
    trust_remote_code=True,
    device_map="auto")

A new version of the following files was downloaded from https://huggingface.co/Lazyhope/RepoSim:
- pipeline.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


[*] Consider setting GitHub token to avoid hitting rate limits. 
For more info, see: https://docs.github.com/authentication/keeping-your-account-and-data-secure/creating-a-personal-access-token


## Turn code and docstring into torch embeddings

In [7]:
import torch

def encode(string):
    with torch.no_grad():
        embedding = model.encode(string, 512)

    return embedding.squeeze().cpu()

registry["code_embeddings"] = registry["code"].apply(encode)
registry["doc_embeddings"] = registry["doc"].apply(encode)
# Convert document embeddings to numpy arrays
registry["doc_embeddings"] = registry["doc_embeddings"].apply(lambda x: np.array(x))
# Convert document embeddings to numpy arrays
registry["code_embeddings"] = registry["code_embeddings"].apply(lambda x: np.array(x))


registry

,code,doc,code_embeddings,doc_embeddings
0,"def add(a, b):\n return a + b",This function adds two numbers.,"[-0.4316707, -0.029330524, 2.6694558, 3.79384,...","[-1.590325, -0.39731324, 3.997744, 2.6590736, ..."
1,"def subtract(a, b):\n return a - b",This function subtracts the second number from...,"[-1.266672, -0.6931733, -0.43216208, 2.7053928...","[-2.4677699, -2.280132, 0.6681174, 1.4311584, ..."
2,"def multiply(a, b):\n return a * b",This function multiplies two numbers.,"[-1.0338337, 1.418475, 1.0938523, 3.0212605, 0...","[-2.3068378, 1.3498034, 3.0785217, 1.3164998, ..."
3,"def divide(a, b):\n return a / b",This function divides the first number by the ...,"[-0.93327546, 2.0973008, 0.7891953, 1.654753, ...","[-3.0390859, -0.07484988, 1.7925602, 0.2812554..."
4,"def power(a, b):\n return a ** b",This function raises the first number to the p...,"[-2.0816395, 1.8067635, 0.6504532, 2.820747, 1...","[-2.946074, -0.51749504, 2.3797052, 2.3707116,..."
5,"def modulus(a, b):\n return a % b",This function returns the remainder when the f...,"[-0.21913469, 0.7265679, -0.20258345, 1.877981...","[-2.490343, -1.3584495, 1.6064605, -0.03004119..."


### Text-to-code Similarity

In [8]:
# Compute user query embeddings
user_query_docs = "Function that adds two numbers"
user_query_docs_emb = encode(user_query_docs)



# Compute cosine similarity
user_query_emb = np.array(user_query_docs_emb)
cos_similarities = cosine_similarity(user_query_emb.reshape(1, -1), np.vstack(registry["doc_embeddings"]))

# Add cosine similarity scores as a new column
registry_doc=registry.copy()
registry_doc["cosine_similarity_doc"] = cos_similarities[0]

# Sort the dataframe based on cosine similarity
sorted_df = registry_doc.sort_values(by="cosine_similarity_doc", ascending=False)

# Retrieve the top 5 most similar documents
top_5_similar_docs = sorted_df.head(5)

In [9]:
top_5_similar_docs

,code,doc,code_embeddings,doc_embeddings,cosine_similarity_doc
0,"def add(a, b):\n return a + b",This function adds two numbers.,"[-0.4316707, -0.029330524, 2.6694558, 3.79384,...","[-1.590325, -0.39731324, 3.997744, 2.6590736, ...",0.976803
2,"def multiply(a, b):\n return a * b",This function multiplies two numbers.,"[-1.0338337, 1.418475, 1.0938523, 3.0212605, 0...","[-2.3068378, 1.3498034, 3.0785217, 1.3164998, ...",0.704091
1,"def subtract(a, b):\n return a - b",This function subtracts the second number from...,"[-1.266672, -0.6931733, -0.43216208, 2.7053928...","[-2.4677699, -2.280132, 0.6681174, 1.4311584, ...",0.700667
3,"def divide(a, b):\n return a / b",This function divides the first number by the ...,"[-0.93327546, 2.0973008, 0.7891953, 1.654753, ...","[-3.0390859, -0.07484988, 1.7925602, 0.2812554...",0.579780
5,"def modulus(a, b):\n return a % b",This function returns the remainder when the f...,"[-0.21913469, 0.7265679, -0.20258345, 1.877981...","[-2.490343, -1.3584495, 1.6064605, -0.03004119...",0.545701


### Code-to-Text

In [10]:
# Compute user query embeddings
user_query_code = "def add_numbers(a, b):\n return a +"
user_query_code_emb = encode(user_query_code)

# Compute cosine similarity
user_query_emb_c = np.array(user_query_code_emb)
cos_similarities = cosine_similarity(user_query_emb_c.reshape(1, -1), np.vstack(registry["code_embeddings"]))

# Add cosine similarity scores as a new column
registry_code=registry.copy()
registry_code["cosine_similarity_code"] = cos_similarities[0]

# Sort the dataframe based on cosine similarity
sorted_df_code = registry_code.sort_values(by="cosine_similarity_code", ascending=False)

# Retrieve the top 5 most similar documents
top_5_similar_code = sorted_df_code.head(5)

In [11]:
top_5_similar_code

,code,doc,code_embeddings,doc_embeddings,cosine_similarity_code
0,"def add(a, b):\n return a + b",This function adds two numbers.,"[-0.4316707, -0.029330524, 2.6694558, 3.79384,...","[-1.590325, -0.39731324, 3.997744, 2.6590736, ...",0.892051
2,"def multiply(a, b):\n return a * b",This function multiplies two numbers.,"[-1.0338337, 1.418475, 1.0938523, 3.0212605, 0...","[-2.3068378, 1.3498034, 3.0785217, 1.3164998, ...",0.618545
1,"def subtract(a, b):\n return a - b",This function subtracts the second number from...,"[-1.266672, -0.6931733, -0.43216208, 2.7053928...","[-2.4677699, -2.280132, 0.6681174, 1.4311584, ...",0.512369
5,"def modulus(a, b):\n return a % b",This function returns the remainder when the f...,"[-0.21913469, 0.7265679, -0.20258345, 1.877981...","[-2.490343, -1.3584495, 1.6064605, -0.03004119...",0.489054
3,"def divide(a, b):\n return a / b",This function divides the first number by the ...,"[-0.93327546, 2.0973008, 0.7891953, 1.654753, ...","[-3.0390859, -0.07484988, 1.7925602, 0.2812554...",0.423482


# Code Sumarization

In [12]:
from transformers import RobertaTokenizer, T5ForConditionalGeneration

tokenizer = RobertaTokenizer.from_pretrained('Salesforce/codet5-base-multi-sum')
model = T5ForConditionalGeneration.from_pretrained('Salesforce/codet5-base-multi-sum')


In [13]:
def generate_summary(text):
    input_ids = tokenizer.encode(text, return_tensors="pt")
    generated_ids = model.generate(input_ids, max_length=20)
    summary = tokenizer.decode(generated_ids[0], skip_special_tokens=True)
    return summary

In [14]:
registry_summary=registry.copy()
# Assuming the registry dataframe is already defined
registry_summary["summarization"] = registry["code"].apply(generate_summary)
registry_summary


,code,doc,code_embeddings,doc_embeddings,summarization
0,"def add(a, b):\n return a + b",This function adds two numbers.,"[-0.4316707, -0.029330524, 2.6694558, 3.79384,...","[-1.590325, -0.39731324, 3.997744, 2.6590736, ...",Add two vectors.
1,"def subtract(a, b):\n return a - b",This function subtracts the second number from...,"[-1.266672, -0.6931733, -0.43216208, 2.7053928...","[-2.4677699, -2.280132, 0.6681174, 1.4311584, ...",Subtract two vectors.
2,"def multiply(a, b):\n return a * b",This function multiplies two numbers.,"[-1.0338337, 1.418475, 1.0938523, 3.0212605, 0...","[-2.3068378, 1.3498034, 3.0785217, 1.3164998, ...",Multiply two vectors.
3,"def divide(a, b):\n return a / b",This function divides the first number by the ...,"[-0.93327546, 2.0973008, 0.7891953, 1.654753, ...","[-3.0390859, -0.07484988, 1.7925602, 0.2812554...",Divide two numbers.
4,"def power(a, b):\n return a ** b",This function raises the first number to the p...,"[-2.0816395, 1.8067635, 0.6504532, 2.820747, 1...","[-2.946074, -0.51749504, 2.3797052, 2.3707116,...",Returns a power of b.
5,"def modulus(a, b):\n return a % b",This function returns the remainder when the f...,"[-0.21913469, 0.7265679, -0.20258345, 1.877981...","[-2.490343, -1.3584495, 1.6064605, -0.03004119...",Returns a % b.
